# Data Mining Project - Group XX 2025/2026

# Import Libraries

In [13]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

from itertools import product
from ydata_profiling import ProfileReport

# for better resolution plots
%config InlineBackend.figure_format = 'retina'

#o svg consegue ampliar infinitamente os gráficos sem perder qualidade mas às vezes é mais lento 
#por isso agora usamos retina


sns.set()

# Loading the Data

Import the datasets from csv files using commas as separators of the columns and setting the unique customer identifier as the index of both columns.

In [14]:
flightsDB = pd.read_csv('data/DM_AIAI_FlightsDB.csv', sep = ",", index_col= "Loyalty#")
customerDB = pd.read_csv('data/DM_AIAI_CustomerDB.csv', sep = ",", index_col= "Loyalty#")
metaData = pd.read_csv('data/DM_AIAI_Metadata.csv', sep = ";", header= None)

Remove the 'Unnamed' column referring to a sequential numbering of the rows, as we set the column "Loyalty#" as the index

In [15]:
customerDB = customerDB.iloc[:, 1:]
customerDB

,First Name,Last Name,Customer Name,Country,Province or State,City,Latitude,Longitude,Postal code,Gender,Education,Location Code,Income,Marital Status,LoyaltyStatus,EnrollmentDateOpening,CancellationDate,Customer Lifetime Value,EnrollmentType
Loyalty#,,,,,,,,,,,,,,,,,,,
480934,Cecilia,Householder,Cecilia Householder,Canada,Ontario,Toronto,43.653225,-79.383186,M2Z 4K1,female,Bachelor,Urban,70146.0,Married,Star,2/15/2019,NaN,3839.14,Standard
549612,Dayle,Menez,Dayle Menez,Canada,Alberta,Edmonton,53.544388,-113.490930,T3G 6Y6,male,College,Rural,0.0,Divorced,Star,3/9/2019,NaN,3839.61,Standard
429460,Necole,Hannon,Necole Hannon,Canada,British Columbia,Vancouver,49.282730,-123.120740,V6E 3D9,male,College,Urban,0.0,Single,Star,7/14/2017,1/8/2021,3839.75,Standard
608370,Queen,Hagee,Queen Hagee,Canada,Ontario,Toronto,43.653225,-79.383186,P1W 1K4,male,College,Suburban,0.0,Single,Star,2/17/2016,NaN,3839.75,Standard
530508,Claire,Latting,Claire Latting,Canada,Quebec,Hull,45.428730,-75.713364,J8Y 3Z5,male,Bachelor,Suburban,97832.0,Married,Star,10/25/2017,NaN,3842.79,2021 Promotion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100012,Ethan,Thompson,Ethan Thompson,Canada,Quebec,Quebec City,46.759733,-71.141009,Y0C 7D6,male,Bachelor,Suburban,NaN,Single,Star,2/27/2019,2/27/2019,NaN,Standard
100013,Layla,Young,Layla Young,Canada,Alberta,Edmonton,53.524829,-113.546357,L3S 9Y3,female,Bachelor,Rural,NaN,Married,Star,9/20/2017,9/20/2017,NaN,Standard
100014,Amelia,Bennett,Amelia Bennett,Canada,New Brunswick,Moncton,46.051866,-64.825428,G2S 2B6,male,Bachelor,Rural,NaN,Married,Star,11/28/2020,11/28/2020,NaN,Standard


# Metadata

**FlightsDB Database Variable Description**
- **Loyalty#:**	Unique customer identifier linking to CustomerDB
- **Year:**	Year of flight activity record
- **Month:**	Month of flight activity record (1-12)
- **YearMonthDate:**	First day of the month for the activity period
- **NumFlights:**	Total number of flights taken by customer in the month
- **NumFlightsWithCompanions:**	Number of flights where customer traveled with companions
- **DistanceKM:**	Total distance traveled in kilometers for the month
- **PointsAccumulated:**	Loyalty points earned by customer during the month
- **PointsRedeemed:**	Loyalty points spent/redeemed by customer during the month
- **DollarCostPointsRedeemed:**	Dollar value of points redeemed during the month

**CustomerDB Database Variable Description**
- **Loyalty#:**  Unique customer identifier for loyalty program members
- **First Name:**   Customer's first name
- **Last Name:**   Customer's last name 
- **Customer Name:** Customer's full name (concatenated)
- **Country:**	Customer's country of residence
- **Province or State:**	Customer's province or state
- **City:**	Customer's city of residence
- **Latitude:**	Geographic latitude coordinate of customer location
- **Longitude:**	Geographic longitude coordinate of customer locatio
- **Postal code:**	Customer's postal/ZIP code
- **Gender:**	Customer's gender
- **Education:**	Customer's highest education level (Bachelor, College, etc.)
- **Location:** Code	Urban/Suburban/Rural classification of customer residence
- **Income:**	Customer's annual income
- **Marital Status:**	Customer's marital status (Married, Single, Divorced)
- **LoyaltyStatus:**	Current tier status in loyalty program (Star > Nova > Aurora)
- **EnrollmentDateOpening:**	Date when customer joined the loyalty program
- **CancellationDate:**	Date when customer left the program
- **Customer Lifetime:** Value	Total calculated monetary value of customer relationship
- **EnrollmentType:**	Method of joining loyalty program

# Data Preparation

## Feature Engineering (FlightsDB)

We will add the following features to our model in order to better analyse the customers and hopefully better segment them in the future.

- `Outstanding_Points:` Total points a customer has accumulated but not yet redeemed. Calculated as TotalPointsAccumulated - TotalPointsRedeemed. Measures potential future redemption and loyalty program “balance”.

- `PointsRedemptionRatio:` The proportion of earned points that a customer has redeemed. Calculated as PointsRedeemed / PointsAccumulated. Shows engagement with the loyalty program and redemption behavior.

- `HighValueTraveler:` Boolean indicating if a customer is a high-value traveler based on total distance. Calculated as DistanceKM > threshold (e.g., 75th percentile). Identifies top-tier travelers for segmentation or targeting.

- `AvgDistancePerFlight:` Average distance traveled per flight. Calculated as DistanceKM / NumFlights. Differentiates between short-haul and long-haul travelers.

- `AvgFlightsPerMonth:` Average number of flights a customer takes per month. Calculated as groupby(Loyalty#)['NumFlights'].mean(). Captures overall travel frequency and consistency.

- `AvgPointsPerFlight:` Average points earned per flight. Calculated as PointsAccumulated / NumFlights. Measures how much loyalty value each flight generates.

- `AvgPointsRedeemedPerFlight:` Average points redeemed per flight. Calculated as PointsRedeemed / NumFlights. Indicates redemption intensity relative to travel.

- `FlightWithCompanionRatio:` Ratio of flights where the customer traveled with companions. Calculated as NumFlightsWithCompanions / NumFlights. Reveals travel habits for group or family targeting.

- `FlightsLast6Months:` Total flights a customer took in the last six months. Calculated as sum of NumFlights in the last 6 months. Captures recent activity and can indicate engagement or churn risk.

- `FlightsVariance:` Variance or standard deviation of number of flights per customer over time. Calculated as groupby(Loyalty#)['NumFlights'].var(). Measures consistency in travel patterns.

- `Recency:` Number of months since a customer’s last flight. Calculated as latest_date - last flight date in months. Key for RFM analysis and engagement assessment.

- `PointsAccPerFlight:` Average points accumulated per flight. Calculated as PointsAccumulated / NumFlights. Shows efficiency of point earning.

- `FlightsPerPointAccumulated:` Number of flights per point earned. Calculated as NumFlights / PointsAccumulated. Measures efficiency of travel relative to points accumulation.

In [ ]:
# Convert 'YearMonthDate' to datetime
flightsDB['YearMonthDate'] = pd.to_datetime(flightsDB['YearMonthDate'])

# Get the latest date in the dataset
latest_date = flightsDB['YearMonthDate'].max()
# Group by Loyalty# to have easy access to customer-level data
grouped = flightsDB.groupby('Loyalty#')

# Outstanding points
flightsDB['Outstanding_Points'] = flightsDB['PointsAccumulated'] - flightsDB['PointsRedeemed']

# Points redemption ratio
flightsDB['PointsRedemptionRatio'] = flightsDB['PointsRedeemed'] / (flightsDB['PointsAccumulated'] + 1e-6)

# High value traveler (example: 75th percentile threshold)
threshold_distance = flightsDB['DistanceKM'].quantile(0.75)
flightsDB['HighValueTraveler'] = flightsDB['DistanceKM'] > threshold_distance

# Average distance per flight
flightsDB['AvgDistancePerFlight'] = flightsDB['DistanceKM'] / (flightsDB['NumFlights'] + 1e-6)

# Average flights per month
flightsDB['AvgFlightsPerMonth'] = grouped['NumFlights'].mean()

# Average points per flight
flightsDB['AvgPointsPerFlight'] = flightsDB['PointsAccumulated'] / (flightsDB['NumFlights'] + 1e-6)

# Average points redeemed per flight
flightsDB['AvgPointsRedeemedPerFlight'] = flightsDB['PointsRedeemed'] / (flightsDB['NumFlights'] + 1e-6)

# Flight with companion ratio
flightsDB['FlightWithCompanionRatio'] = grouped.apply(lambda x: x['NumFlightsWithCompanions'].sum() / (x['NumFlights'].sum() + 1e-6))

# Flights in last 6 months
six_months_ago = latest_date - pd.DateOffset(months=6)
flights_last_6m = flightsDB[flightsDB['YearMonthDate'] >= six_months_ago].groupby('Loyalty#')['NumFlights'].sum()
flightsDB['FlightsLast6Months'] = flights_last_6m
flightsDB['FlightsLast6Months'].fillna(0, inplace=True)

# Flights variance
flightsDB['FlightsVariance'] = grouped['NumFlights'].var().fillna(0)

# Recency (months since last flight)
last_flight = grouped['YearMonthDate'].max()
flightsDB['Recency'] = (latest_date - last_flight).dt.days / 30
flightsDB['Recency'] = flightsDB['Recency'].astype(int)

# Points per flight
flightsDB['PointsAccPerFlight'] = flightsDB['PointsAccumulated'] / (flightsDB['NumFlights'] + 1e-6)

# Flights per point accumulated
flightsDB['FlightsPerPointAccumulated'] = flightsDB['NumFlights'] / (flightsDB['PointsAccumulated'] + 1e-6)

flightsDB.head()

/var/folders/x4/93zv09_n7r951k9d9t268vt80000gn/T/ipykernel_39588/262508047.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  flightsDB['FlightsLast6Months'].fillna(0, inplace=True)


,Year,Month,YearMonthDate,NumFlights,NumFlightsWithCompanions,DistanceKM,PointsAccumulated,PointsRedeemed,DollarCostPointsRedeemed,Outstanding_Points,...,AvgDistancePerFlight,AvgFlightsPerMonth,AvgPointsPerFlight,AvgPointsRedeemedPerFlight,FlightWithCompanionRatio,FlightsLast6Months,FlightsVariance,Recency,PointsAccPerFlight,FlightsPerPointAccumulated
Loyalty#,,,,,,,,,,,,,,,,,,,,,
413052,2021,12,2021-12-01,2.0,2.0,9384.0,938.0,0.0,0.0,938.0,...,4691.997654,3.550000,468.999766,0.000000,0.318466,34.0,17.214000,0,468.999766,0.002132
464105,2021,12,2021-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,4.975000,0.000000,0.000000,0.353992,62.0,29.141357,0,0.000000,0.000000
681785,2021,12,2021-12-01,10.0,3.0,14745.0,1474.0,0.0,0.0,1474.0,...,1474.499853,5.461111,147.399985,0.000000,0.369278,41.0,28.551587,0,147.399985,0.006784
185013,2021,12,2021-12-01,16.0,4.0,26311.0,2631.0,3213.0,32.0,-582.0,...,1644.437397,4.322222,164.437490,200.812487,0.280848,67.0,23.173206,0,164.437490,0.006081
216596,2021,12,2021-12-01,9.0,0.0,19275.0,1927.0,0.0,0.0,1927.0,...,2141.666429,3.286111,214.111087,0.000000,0.280642,47.0,22.604659,0,214.111087,0.004670


In [8]:
flightsDB.head()

,Year,Month,YearMonthDate,NumFlights,NumFlightsWithCompanions,DistanceKM,PointsAccumulated,PointsRedeemed,DollarCostPointsRedeemed,AvgDistancePerFlight,PropFlightsWithCompanions,NetPoints,DollarPerPoint
Loyalty#,,,,,,,,,,,,,
413052,2021,12,12/1/2021,2.0,2.0,9384.0,938.0,0.0,0.0,4692.000000,1.00,938.0,NaN
464105,2021,12,12/1/2021,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN
681785,2021,12,12/1/2021,10.0,3.0,14745.0,1474.0,0.0,0.0,1474.500000,0.30,1474.0,NaN
185013,2021,12,12/1/2021,16.0,4.0,26311.0,2631.0,3213.0,32.0,1644.437500,0.25,-582.0,0.00996
216596,2021,12,12/1/2021,9.0,0.0,19275.0,1927.0,0.0,0.0,2141.666667,0.00,1927.0,NaN


!!!!!!!!    SE JÁ FOR PARA CRIAR FEATURES USANDO VALORES DE MÉDIA, MEDIANA,.....           !!!!!!!!!!!!

In [9]:
# Transformar YearMonthDate em datetime
flights['YearMonthDate'] = pd.to_datetime(flights['YearMonthDate'])
flights['Month'] = flights['YearMonthDate'].dt.month
flights['Year'] = flights['YearMonthDate'].dt.year

# Exemplo de agregações mensais/anuais por cliente
agg_flights = flights.groupby('Loyalty#').agg({
    'NumFlights': 'sum',
    'NumFlightsWithCompanions': 'sum',
    'DistanceKM': 'sum',
    'PointsAccumulated': 'sum',
    'PointsRedeemed': 'sum',
    'DollarCostPointsRedeemed': 'sum',
    'AvgDistancePerFlight': 'mean',
    'PropFlightsWithCompanions': 'mean',
    'NetPoints': 'sum',
    'DollarPerPoint': 'mean'
}).reset_index()

NameError: name 'flights' is not defined

## Feature Engineering (CustomerDB)

In [ ]:
# transform EnrollmentDateOpening in datetime and calculate CustomerLifetimeMonths
customerDB['EnrollmentDateOpening'] = pd.to_datetime(customerDB['EnrollmentDateOpening'])
customerDB['CustomerLifetimeMonths'] = ((pd.Timestamp.today() - customerDB['EnrollmentDateOpening']).dt.days) / 30

# Codifyar LoyaltyStatus como ordinal
status_map = {'Aurora': 0, 'Nova': 1, 'Star': 2}
customerDB['LoyaltyStatusCode'] = customerDB['LoyaltyStatus'].map(status_map)

# Categorize Income as a categorical variable
income_bins = [0, 40000, 80000, 150000, float('inf')]
income_labels = ['Low', 'Medium', 'High', 'Very High']
customerDB['IncomeCategory'] = pd.cut(customerDB['Income'], bins=income_bins, labels=income_labels)


In [ ]:
customerDB.head()

## Merge Datasets

In [ ]:
df_final = pd.merge(flightsDB, customerDB, on='Loyalty#', how='inner')
df_final.head()

## Feature Engineering combined with both datasets

In [ ]:
df_final['FlightsPerIncome'] = df_final['NumFlights'] / df_final['Income']
df_final['NetPointsPerIncome'] = df_final['NetPoints'] / df_final['Income']
df_final['DistancePerLifetime'] = df_final['DistanceKM'] / df_final['CustomerLifetimeMonths']

In [ ]:
df_final.head()